<img src='https://media.istockphoto.com/vectors/covid19-outbreak-impacting-the-stock-market-markets-plunging-economic-vector-id1209967012' style='height:400px'>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_covid_basic = pd.read_csv('/kaggle/input/covid19dataset/COVID_Data_Basic.csv')
df_covid = pd.read_csv('/kaggle/input/covid19dataset/COVID_Data.csv')
df_ibov = pd.read_csv('/kaggle/input/ibovespa-points/datasets_Ibovespa_points_1994_2020.csv')

In [ ]:
df_brazil_covid = df_covid_basic[df_covid_basic['Country']=='Brazil']
def convert_date(date):
    splitted_date = date.split(".")
    return splitted_date[2] + "-"+ splitted_date[1] + "-" + splitted_date[0]

def remove_m_volume(str):
    if 'M' in str:
        str = str.replace("M",'')
        str = str.replace(',','')
        return float(str) * 1000000
    elif 'K' in str:
        str = str.replace("K",'')
        str = str.replace(',','')
        return float(str) * 1000
    else:
        return 0
    
df_ibov['Date'] = df_ibov['Date'].apply(lambda x : convert_date(x))
df_ibov['Volume'] = df_ibov['Volume'].apply(lambda x : remove_m_volume(x))
df_ibov['Volume'] = df_ibov['Volume'].astype(float)

df_covid_period_ibov = df_ibov[df_ibov['Date']>='2020-01-01']
df_brazil_covid = df_brazil_covid[df_brazil_covid['Date'].isin(list(df_covid_period_ibov['Date']))]
df_covid_period_ibov = df_covid_period_ibov.drop(df_covid_period_ibov.index[[0,1]])
df_brazil_covid = df_brazil_covid.sort_values(by='Date', ascending=True)
df_covid_period_ibov = df_covid_period_ibov.sort_values(by='Date', ascending=True)
df_covid_period_ibov['Close points'] = df_covid_period_ibov['Close points'].apply(lambda x : x.replace(',','.'))
df_covid_period_ibov['Close points'] = df_covid_period_ibov['Close points'].astype(float)

<div class="list-group" id="list-tab" role="tablist">
  <h1 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">&nbsp;Summary:</h1>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#1" role="tab" aria-controls="profile">1. Introduction<span class="badge badge-primary badge-pill">1</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#2" role="tab" aria-controls="messages">2. Time Series<span class="badge badge-primary badge-pill">2</span></a>
      <a class="list-group-item list-group-item-action" data-toggle="list" href="#2" role="tab" aria-controls="messages">3. Correlations<span class="badge badge-primary badge-pill">3</span></a>
      <a class="list-group-item list-group-item-action" data-toggle="list" href="#2" role="tab" aria-controls="messages">4. Conclusion<span class="badge badge-primary badge-pill">4</span></a>
</div>

<a id="1"></a> <br>
<font size="+3" color="black"><b>1 - Introduction</b></font><br><a id="1"></a>
<br> 

* lets see the brazilian stock market and covids charts

In [ ]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import datetime
import plotly.express as px
from plotly.graph_objs import *
import plotly.graph_objects as go

def plot_line_chart(df, date_column1, column2, title):
    fig = px.line(x=pd.to_datetime(df[date_column1].values), y=df[column2])
    fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })
    fig.show()

def covid_charts(df, column1, column2, title):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['Date'].values, y=df[column1],
        name=column1,
        mode='lines',
        marker_color='rgba(152, 0, 0, .8)'
    ))

    fig.add_trace(go.Scatter(
        x=df['Date'].values, y=df[column2],
        name=column2,
        mode='lines',
        marker_color='rgba(255, 182, 193, .9)'
    ))
    fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })
    fig.show()

plot_line_chart(df_covid_period_ibov, 'Date', 'Close points', 'Ibovespa index in covid period')
covid_charts(df_brazil_covid,'Confirmed', 'Death', 'Confirmed cases vs Deaths')
covid_charts(df_brazil_covid,'newConfirmed', 'newDeath', 'Daily confirmed cases vs Daily new deaths')

* First chart ilustrate the impact of covid period in IBovespa index, compare it with others wen can observer that in the same period when the are low number of cases the impact over index is great, but when the contamination evolves and deaths grow up the index start recovery.

* Lets observer how stock brazilian market Ibovespa behaved over the first 6 months of the corona virus pandemic

In [ ]:
def plot_line_chart_rolling_mensure(df, column2, title):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['Date'].values, y=df[column2].rolling(window=7, center=False).mean(),
        name='rolling mean '+ column2,
        mode='lines',
        marker_color='rgba(255, 0, 0, .8)'
    ))
    fig.add_trace(go.Scatter(
        x=df['Date'].values, y=df[column2].rolling(window=7, center=False).std(),
        name='rolling std ' + column2,
        mode='lines',
        marker_color='rgba(0, 255, 0, .9)'
    ))
    fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })
    fig.show()
    
plot_line_chart_rolling_mensure(df_covid_period_ibov, 'Close points', 'Ibovespa index in covid period')

<a id="2"></a> <br>
<font size="+3" color="black"><b>2 - Time Series</b></font><br><a id="2"></a>
<br> 

* the graph above shows that there was a period of strong volatility where the point average drops a lot and the standard deviation goes up
* so lets try to build a model that predict this difficult time.
* First to generate this model we need a stationary time series 
  - stationatiry time serie has some properties like:
      - constant mean 
      - constant variance

* our current series apparently dont have this properties so to confirme this i will apply adf_test if p-value is 0 or to next 0 we can accept test

In [ ]:
from statsmodels.tsa.stattools import adfuller
def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

adf_test(df_covid_period_ibov['Close points'].values)

* so our test failed, know i need to convert this series to a stationary series we can do this by the deseasonality or detrend, i will apply the two methods and select series the better fits the adf test

In [ ]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return diff

detrend = difference(df_covid_period_ibov['Close points'].values)
frame = pd.DataFrame([])
frame['detrend'] = detrend

fig = go.Figure()
fig.add_trace(go.Scatter(
    y=frame['detrend'].values,
    name='rolling mean ',
    mode='lines',
    marker_color='rgba(0, 0, 255, .8)'
))
fig.add_trace(go.Scatter(
    y=frame['detrend'].rolling(window=7, center=False).mean(),
    name='rolling mean ',
    mode='lines',
    marker_color='rgba(255, 0, 0, .8)'
))
fig.add_trace(go.Scatter(
    y=frame['detrend'].rolling(window=7, center=False).std(),
    name='rolling std ' ,
    mode='lines',
    marker_color='rgba(0, 255, 0, .9)'
))
fig.update_layout({
    'title': 'detrend series',
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

adf_test(detrend)

* The results looklike good we have a really small value to p-value with the detrend method to convert the series to stationary
* Now use the seasonality to see whats happen in adf_test

In [ ]:
deseasonality = difference(df_covid_period_ibov['Close points'].values, 7)
frame = pd.DataFrame([])
frame['detrend'] = deseasonality

fig = go.Figure()
fig.add_trace(go.Scatter(
    y=frame['detrend'].values,
    name='rolling mean ',
    mode='lines',
    marker_color='rgba(0, 0, 255, .8)'
))
fig.add_trace(go.Scatter(
    y=frame['detrend'].rolling(window=7, center=False).mean(),
    name='rolling mean ',
    mode='lines',
    marker_color='rgba(255, 0, 0, .8)'
))
fig.add_trace(go.Scatter(
    y=frame['detrend'].rolling(window=7, center=False).std(),
    name='rolling std ' ,
    mode='lines',
    marker_color='rgba(0, 255, 0, .9)'
))
fig.update_layout({
    'title': 'deseasonality series',
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

adf_test(deseasonality)

* The seasonality gave a worse result, so the series the i will use is product of detrend 
* To modelling the series i will use arima model, in arima we need pass two params 'p' and 'q'
  - p : is the value choose to AR model  terms are lags of dependent variable  
  - q : is the value of MA model are lags of erros in predict equation
  
* To determine this values we use the acf and pacf 

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
acf_values = acf(detrend)
pacf_values = pacf(detrend, method='ols')

In [ ]:
def compute_limit(serie, upper=True):
    limit = []
    y=1.96/np.sqrt(len(serie))
    for i in range(0, len(serie)):
        if upper == True:
            limit.append(y)
        else:
            limit.append(-y)
    return limit

def plot_acf(serie, title):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=serie,
        name='serie',
        mode='lines',
        marker_color='rgba(0, 0, 255, .8)'
    ))
    fig.add_trace(go.Scatter(
        y=compute_limit(serie),
        name='upper limit',
        mode='lines',
        marker_color='rgba(0, 0, 0, .8)'
    ))
    fig.add_trace(go.Scatter(
        y=compute_limit(serie, False),
        name='down limit' ,
        mode='lines',
        marker_color='rgba(0, 0, 0, .9)'
    ))
    fig.update_layout({
        'title': title,
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })
    fig.show()  
plot_acf(acf_values, 'Autocorrelation function')

In [ ]:
plot_acf(pacf_values, 'Partial Autocorrelation function')

* the chosen values of acf and pacf are the points on the X axis where the blue line touches the black line

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(df_covid_period_ibov['Close points'].values, order=(1,0,1)) 
model_fit = model.fit(disp=0)

In [ ]:
from pandas import datetime
forecast = model_fit.predict()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
        y=forecast,
        name='predict',
        mode='lines',
        marker_color='rgba(0, 0, 255, .8)'
))
fig.add_trace(go.Scatter(
        y=df_covid_period_ibov['Close points'].values,
        name='Original',
        mode='lines',
        marker_color='rgba(0, 0, 0, .8)'
))
fig.update_layout({
        'title': 'Predict vs Real',
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show() 

* as shown in the graph above the predicted values ​​are very close to the actual values

<a id="3"></a> <br>
<font size="+3" color="black"><b>3 - Correlations</b></font><br><a id="3"></a>
<br> 

In [ ]:
import seaborn as sns

def heatmap(df, df2,column1, column2, title): 
    matrix = np.corrcoef(df[column1].values, df2[column2].values)
    ax = sns.heatmap(matrix, annot=True, fmt="f", 
                     yticklabels=[column1, column2], 
                     xticklabels=[column1, column2])
    ax.set_title(title)

heatmap(df_covid_period_ibov, df_brazil_covid, 'Close points', 'Death', 'correlation between Ibovespa points and Death by corona')

In [ ]:
heatmap(df_covid_period_ibov, df_brazil_covid, 'Close points', 'Confirmed', 
        'correlation between Ibovespa points and total confirmed cases')

In [ ]:
heatmap(df_covid_period_ibov, df_brazil_covid, 'Close points', 'newDeath', 'correlation between Ibovespa points and daily deaths')

In [ ]:
heatmap(df_covid_period_ibov, df_brazil_covid, 'Close points', 'newConfirmed', 'correlation between Ibovespa points and daily new cases')

* Unlike the first set of charts ilustrates, that how deaths and confirmed cases grow up the Ibovespa index grow up too, the correlation among daily, total of deaths and confirmed cases, show that this variables has negative impact of brazilian stock market Ibovespa.

In [ ]:
heatmap(df_covid_period_ibov, df_brazil_covid, 'Close points', 'Recovered', 'correlation between Ibovespa points and recover')

In [ ]:
heatmap(df_covid_period_ibov, df_brazil_covid, 'Close points', 'newRecovered', 'correlation between Ibovespa points and daily recovered')

* How people are recovered of covid,total and daily demonstrate dont impact index positively or negatively.

<a id="4"></a> <br>
<font size="+3" color="black"><b>4 - Conclusion</b></font><br><a id="4"></a>
<br> 
* Maybe in the short term the covid impact in brazilian stock market is speculative because when the first deaths group up, and when low contamination and 0 deaths index fell sharply, but when we observer the correlation between death, cases and recovery, is possible look that are some relationship but not enough to define the behavior of the index.

* working in progress